In [1]:
# Notebook 2x: Generate IDENTICAL Questions for Hindi and Chinese - Apples-to-Apples Comparison
# Run this BEFORE notebook 3 to create identical questions in both languages

import os
import re
import json
import random
import pandas as pd
from pathlib import Path
from collections import Counter, defaultdict
from typing import List, Dict
import openai
from pprint import pformat

# API KEY
OPEN_API_KEY = "YOUR KEY"
os.environ["OPENAI_API_KEY"] = OPEN_API_KEY
openai.api_key = OPEN_API_KEY

# Correct v1 client
from openai import OpenAI
client = OpenAI(api_key=OPEN_API_KEY)

# Paths
ROOT_DIR = Path('..').resolve()
RAW_DATA_DIR = ROOT_DIR / 'data' / 'raw'
PROCESSED_DATA_DIR = ROOT_DIR / 'data' / 'processed'
QUESTIONS_DIR = ROOT_DIR / 'data' / 'questions'
QUESTIONS_DIR.mkdir(parents=True, exist_ok=True)

print("="*80)
print("GENERATING IDENTICAL QUESTIONS FOR HINDI AND CHINESE")
print("Strategy: Generate English questions, then translate to both languages")
print("="*80)

# Load processed documents
with open(PROCESSED_DATA_DIR / 'processed_documents.json', 'r', encoding='utf-8') as f:
    docs_data = json.load(f)

print(f"\nTotal documents: {len(docs_data)}")

# Analyze by source
sources = Counter([doc['metadata']['source'] for doc in docs_data])
print(f"\nDocuments by source:")
for source, count in sources.items():
    print(f"  {source}: {count} documents")

# Sample content from each source
print("\n" + "="*80)
print("CORPUS ANALYSIS FOR QUESTION GENERATION")
print("="*80)

for source in sources.keys():
    source_docs = [doc for doc in docs_data if doc['metadata']['source'] == source]
    if not source_docs:
        continue
    print(f"\n{source} - Sample content:")
    print((source_docs[0].get('content') or "")[:500] + "...")

# Generate base English questions
random.seed(42)

def _sample_corpus(docs_data, per_source=3, max_sources=4, clip_chars=600):
    """Small, diverse, token-safe corpus sample."""
    uniq_sources = list({doc["metadata"]["source"] for doc in docs_data})
    random.shuffle(uniq_sources)
    uniq_sources = uniq_sources[:max_sources]

    samples = []
    for src in uniq_sources:
        source_docs = [d for d in docs_data if d["metadata"]["source"] == src]
        if not source_docs:
            continue
        random.shuffle(source_docs)
        take = min(per_source, len(source_docs))
        for d in source_docs[:take]:
            content = (d.get("content") or "")[:clip_chars]
            samples.append({"source": src, "content": content})
    return samples

def _normalize_category(cat: str) -> str:
    c = (cat or "").strip().lower()
    if c in {"medication","condition","treatment","side_effect"}:
        return c
    if "side" in c: return "side_effect"
    if "treat" in c: return "treatment"
    if "cond" in c: return "condition"
    return "medication"

def _normalize_source(src: str) -> str:
    s = (src or "").lower()
    if "cdc" in s: return "CDC"
    if "medline" in s or "nih" in s: return "MedlinePlus (NIH)"
    return "FDA"

def generate_base_english_questions(docs_data: List[Dict], num_questions: int = 30) -> List[Dict]:
    """Generate base English questions that will be translated to both languages"""
    samples = _sample_corpus(docs_data, per_source=3, max_sources=4, clip_chars=600)
    if not samples:
        print("[Question gen] No samples found; returning empty list.")
        return []

    corpus_context = "\n\n---SAMPLE FROM CORPUS---\n\n".join(
        [f"Source: {s['source']}\n{s['content']}" for s in samples]
    )

    prompt = f"""You are a medical question generator. Based ONLY on the corpus samples below, generate EXACTLY {num_questions} English medical questions that CAN BE ANSWERED from these samples.

CORPUS SAMPLES:
{corpus_context}

RULES:
1) Every question must be answerable using the samples above (no outside knowledge).
2) Cover medications, conditions, treatments, and side effects actually present.
3) Generate questions in ENGLISH only (we will translate them later).
4) category ∈ ["medication","condition","treatment","side_effect"].
5) complexity ∈ ["simple","moderate","complex"].
6) expected_source ∈ ["FDA","CDC","MedlinePlus (NIH)"] chosen from the samples' sources.
7) Respond ONLY as a JSON OBJECT with a top-level key "questions" whose value is a JSON ARRAY of exactly {num_questions} items.
8) No extra keys or commentary outside "questions".

JSON SHAPE:
{{
  "questions": [
    {{
      "english": "string",
      "category": "medication|condition|treatment|side_effect",
      "complexity": "simple|moderate|complex",
      "expected_source": "FDA|CDC|MedlinePlus (NIH)",
      "answerable": true
    }}
  ]
}}
"""

    raw = None
    try:
        resp = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You generate answerable medical questions strictly grounded in the provided corpus."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            response_format={"type": "json_object"},
        )
        raw = resp.choices[0].message.content
        data = json.loads(raw)
        qs = data.get("questions", [])
    except Exception as e:
        print(f"[Question gen] Error parsing json_object: {e}")
        try:
            resp2 = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "Output ONLY valid JSON per the user instruction."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.5,
            )
            raw = resp2.choices[0].message.content
            m = re.search(r"\{.*\}", raw, flags=re.S)
            data = json.loads(m.group(0)) if m else json.loads(raw)
            qs = data.get("questions", data if isinstance(data, list) else [])
        except Exception as e2:
            print(f"[Question gen] Fallback parse failed: {e2}")
            if raw:
                print("[Assistant raw output]:\n", raw[:2000])
            return []

    out = []
    for q in qs:
        if not isinstance(q, dict):
            continue
        en = (q.get("english") or "").strip()
        if not en:
            continue
        out.append({
            "english": en,
            "category": _normalize_category(q.get("category")),
            "complexity": (q.get("complexity") or "simple").strip().lower(),
            "expected_source": _normalize_source(q.get("expected_source")),
            "answerable": True
        })

    return out[:num_questions]

def translate_questions_to_language(questions: List[Dict], target_language: str, language_name: str) -> List[Dict]:
    """Translate English questions to target language"""
    print(f"\nTranslating questions to {language_name}...")
    
    translated_questions = []
    
    # Process in batches of 5 for efficiency
    batch_size = 5
    for i in range(0, len(questions), batch_size):
        batch = questions[i:i+batch_size]
        
        # Prepare batch for translation
        batch_text = "\n".join([f"{j+1}. {q['english']}" for j, q in enumerate(batch)])
        
        prompt = f"""Translate the following medical questions to {language_name}. Maintain medical accuracy and natural fluency.

English Questions:
{batch_text}

Instructions:
1. Translate each question accurately to {language_name}
2. Maintain the medical terminology appropriately
3. Keep the same question numbering
4. Respond with ONLY the translated questions, one per line, with numbers

Example format:
1. [Translated question 1]
2. [Translated question 2]
etc.
"""

        try:
            resp = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": f"You are an expert medical translator for {language_name}. Translate accurately while maintaining medical precision."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
            )
            
            translated_text = resp.choices[0].message.content
            translated_lines = [line.strip() for line in translated_text.split('\n') if line.strip()]
            
            # Extract translations
            for j, q in enumerate(batch):
                if j < len(translated_lines):
                    # Remove numbering (e.g., "1. " from the start)
                    translation = re.sub(r'^\d+\.\s*', '', translated_lines[j])
                    
                    translated_q = q.copy()
                    translated_q[target_language] = translation
                    translated_questions.append(translated_q)
                    
                    print(f"  {i+j+1:2d}. {translation}")
                else:
                    # Fallback if translation failed
                    translated_q = q.copy()
                    translated_q[target_language] = f"[Translation failed for: {q['english']}]"
                    translated_questions.append(translated_q)
        
        except Exception as e:
            print(f"Error translating batch {i//batch_size + 1}: {e}")
            # Add fallback entries
            for q in batch:
                translated_q = q.copy()
                translated_q[target_language] = f"[Translation failed for: {q['english']}]"
                translated_questions.append(translated_q)
    
    return translated_questions

# Validate questions against corpus
def validate_question_answerability(question_english: str, docs_data: List[Dict]) -> Dict:
    """Check if a question can likely be answered from the corpus"""
    key_terms = re.findall(r'\b\w+\b', question_english.lower())
    key_terms = [t for t in key_terms if len(t) > 4]

    relevant_docs = []
    for doc in docs_data:
        content_lower = (doc.get('content') or "").lower()
        matches = sum(1 for term in key_terms if term in content_lower)
        if matches >= 2:
            relevant_docs.append({
                'content': (doc.get('content') or "")[:300],
                'source': doc['metadata']['source'],
                'match_count': matches
            })

    return {
        'answerable': len(relevant_docs) > 0,
        'relevant_docs_count': len(relevant_docs),
        'top_docs': sorted(relevant_docs, key=lambda x: x['match_count'], reverse=True)[:3]
    }

print("\n" + "="*80)
print("STEP 1: GENERATING BASE ENGLISH QUESTIONS")
print("="*80)

# Generate base English questions
base_questions = generate_base_english_questions(docs_data, num_questions=30)
print(f"Generated {len(base_questions)} base English questions")

# Show samples
print(f"\nSample English questions:")
for i, q in enumerate(base_questions[:5], 1):
    print(f"  {i}. {q['english']}")
    print(f"     Category: {q['category']} | Complexity: {q['complexity']}")

print("\n" + "="*80)
print("STEP 2: VALIDATING QUESTIONS AGAINST CORPUS")
print("="*80)

# Validate all questions
validated_questions = []
for q in base_questions:
    validation = validate_question_answerability(q['english'], docs_data)
    q['validation'] = validation
    if validation['answerable']:
        validated_questions.append(q)

print(f"Answerable questions: {len(validated_questions)}/{len(base_questions)}")

# If we don't have enough, take the best ones we have
if len(validated_questions) < 30:
    print(f"Warning: Only {len(validated_questions)} questions are fully answerable from corpus")
    print("Using all available validated questions")
    final_questions = validated_questions
else:
    # Take exactly 30
    final_questions = validated_questions[:30]

print(f"Final question set: {len(final_questions)} questions")

print("\n" + "="*80)
print("STEP 3: TRANSLATING TO HINDI AND CHINESE")
print("="*80)

# Translate to Hindi
hindi_questions = translate_questions_to_language(final_questions, "hindi", "Hindi")
print(f"\nCompleted Hindi translations: {len(hindi_questions)}")

# Translate to Chinese
chinese_questions = translate_questions_to_language(final_questions, "chinese", "Chinese")
print(f"Completed Chinese translations: {len(chinese_questions)}")

print("\n" + "="*80)
print("STEP 4: ORGANIZING BY CATEGORY AND SAVING")
print("="*80)

# Group by category for both languages
def group_by_category(questions):
    grouped = defaultdict(list)
    for q in questions:
        grouped[q['category']].append(q)
    return dict(grouped)

hindi_grouped = group_by_category(hindi_questions)
chinese_grouped = group_by_category(chinese_questions)

# Save to JSON files (NO TIMESTAMPS)
hindi_file = QUESTIONS_DIR / 'hindi_questions.json'
chinese_file = QUESTIONS_DIR / 'chinese_questions.json'

with open(hindi_file, 'w', encoding='utf-8') as f:
    json.dump(hindi_grouped, f, indent=2, ensure_ascii=False)

with open(chinese_file, 'w', encoding='utf-8') as f:
    json.dump(chinese_grouped, f, indent=2, ensure_ascii=False)

print(f"\nSaved questions to:")
print(f"  Hindi: {hindi_file}")
print(f"  Chinese: {chinese_file}")

# Verification: Show that questions are identical (same English base)
print("\n" + "="*80)
print("VERIFICATION: ENSURING IDENTICAL QUESTION SETS")
print("="*80)

print(f"\nHindi questions by category:")
for category, questions in hindi_grouped.items():
    print(f"  {category}: {len(questions)} questions")

print(f"\nChinese questions by category:")
for category, questions in chinese_grouped.items():
    print(f"  {category}: {len(questions)} questions")

# Verify same English questions
print(f"\nVerifying identical English base:")
hindi_english = set()
chinese_english = set()

for questions in hindi_grouped.values():
    for q in questions:
        hindi_english.add(q['english'])

for questions in chinese_grouped.values():
    for q in questions:
        chinese_english.add(q['english'])

if hindi_english == chinese_english:
    print("✅ SUCCESS: Both language sets have identical English questions")
    print(f"   Total unique English questions: {len(hindi_english)}")
else:
    print("❌ ERROR: Question sets don't match!")
    print(f"   Hindi has {len(hindi_english)} unique questions")
    print(f"   Chinese has {len(chinese_english)} unique questions")

# Show sample comparison
print(f"\nSample question comparison:")
print(f"{'='*80}")

# Get first question from each language for comparison
sample_hindi = hindi_questions[0] if hindi_questions else None
sample_chinese = chinese_questions[0] if chinese_questions else None

if sample_hindi and sample_chinese:
    print(f"English: {sample_hindi['english']}")
    print(f"Hindi:   {sample_hindi['hindi']}")
    print(f"Chinese: {sample_chinese['chinese']}")
    print(f"Category: {sample_hindi['category']} | Complexity: {sample_hindi['complexity']}")

print("\n" + "="*80)
print("FINAL SUMMARY")
print("="*80)

print(f"\n✅ Generated identical question sets for apples-to-apples comparison")
print(f"   Total questions per language: {len(final_questions)}")
print(f"   Languages: Hindi, Chinese")
print(f"   Base: Same English questions translated to both languages")

print(f"\nBreakdown by category:")
all_categories = set()
if hindi_grouped:
    all_categories.update(hindi_grouped.keys())
if chinese_grouped:
    all_categories.update(chinese_grouped.keys())

for category in sorted(all_categories):
    hindi_count = len(hindi_grouped.get(category, []))
    chinese_count = len(chinese_grouped.get(category, []))
    print(f"  {category}: {hindi_count} Hindi, {chinese_count} Chinese")

print("\n" + "="*80)
print("READY FOR NOTEBOOK 3!")
print("="*80)
print(f"\nNext steps:")
print(f"1. Run notebook 3 to load these identical question sets")
print(f"2. Compare Hindi vs Chinese performance on SAME questions")
print(f"3. Ensure true apples-to-apples comparison across languages")
print(f"4. Results will show language-specific differences, not question difficulty differences")

# Save a master comparison file for reference
comparison_data = []
for i, (h_q, c_q) in enumerate(zip(hindi_questions, chinese_questions)):
    comparison_data.append({
        'question_id': f'Q{i+1:03d}',
        'english': h_q['english'],
        'hindi': h_q['hindi'],
        'chinese': c_q['chinese'],
        'category': h_q['category'],
        'complexity': h_q['complexity'],
        'expected_source': h_q['expected_source']
    })

comparison_file = QUESTIONS_DIR / 'question_comparison.json'
with open(comparison_file, 'w', encoding='utf-8') as f:
    json.dump(comparison_data, f, indent=2, ensure_ascii=False)

print(f"\n📋 Saved master comparison file: {comparison_file}")
print(f"   This file shows all questions side-by-side for verification")

GENERATING IDENTICAL QUESTIONS FOR HINDI AND CHINESE
Strategy: Generate English questions, then translate to both languages

Total documents: 16036

Documents by source:
  FDA: 15942 documents
  MedlinePlus (NIH): 94 documents

CORPUS ANALYSIS FOR QUESTION GENERATION

FDA - Sample content:
INDICATIONS AND USAGE: 1 INDICATIONS AND USAGE Naproxen tablets and naproxen sodium tablets are indicated for: the relief of the signs and symptoms of: • rheumatoid arthritis • osteoarthritis • ankylosing spondylitis • Polyarticular Juvenile Idiopathic Arthritis Naproxen tablets and naproxen sodium tablets are also indicated for: the relief of signs and symptoms of: • tendonitis • bursitis • acute gout the management of: • pain • primary dysmenorrhea Naproxen tablets and naproxen sodium tablets ...

MedlinePlus (NIH) - Sample content:
SUMMARY: What is asthma?Asthma is a chronic (long-term) lung disease. It affects your airways, the tubes that carry air in and out of your lungs. When you have asthma, 